## SQL Project
The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers.
You've been given a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

In [1]:
# import libraries
import pandas as pd
!pip install psycopg2-binary
from sqlalchemy import create_engine


db_config = {'user': 'user',         # user name
             'pwd': 'pwd', # password
             'host': 'host', # host
             'port': 6432,              # connection port
             'db': 'db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


def taskfunc(query):
    data=pd.io.sql.read_sql(query, con = engine)
    display(data)





Defaulting to user installation because normal site-packages is not writeable


## 1.  Print the head of all tables

In [2]:
def printfunc(table):
    taskfunc(
    '''
    SELECT * FROM {}


    LIMIT 5
    '''.format(table)
    )

In [3]:
table=["books","ratings","reviews","publishers","authors"]

In [4]:
for tablename in table:
    printfunc(tablename)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
def dtaskfunc(query):
    data=pd.io.sql.read_sql(query, con = engine)

In [6]:
def dprintfunc(table):
    dtaskfunc(
    '''
    SELECT * FROM {}


    
    '''.format(table)
    )

In [7]:
for tablename in table:
    abc=pd.DataFrame(dprintfunc(tablename))
    abc.duplicated()

An empty dataframe show there are no duplicates

## 2.1 Find the number of books released after January 1, 2000.

In [8]:
taskfunc(
'''
SELECT COUNT(*)

FROM books

WHERE
            CAST(books.publication_date AS date)  > '2000-01-01'

LIMIT 5
'''
)

,count
0,819


The books released after 2000 are 819 pieces

## 2.2 Find the number of user reviews and the average rating for each book.

In [9]:
table

['books', 'ratings', 'reviews', 'publishers', 'authors']

In [10]:
taskfunc(
'''
SELECT 
    books.title,
    AVG(ratings.rating),
    COUNT(DISTINCT reviews.username)
FROM
    reviews
    LEFT JOIN 
       ratings ON ratings.book_id = reviews.book_id
    LEFT JOIN 
       books ON books.book_id = reviews.book_id
Group by
books.title
ORDER BY
count DESC




'''
)

,title,avg,count
0,Memoirs of a Geisha,4.138462,8
1,Twilight (Twilight #1),3.662500,7
2,The Da Vinci Code (Robert Langdon #2),3.830508,6
3,The Road,3.772727,6
4,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
...,...,...,...
988,Naked Empire (Sword of Truth #8),3.500000,1
989,Moo Baa La La La!,3.000000,1
990,Merrick (The Vampire Chronicles #7),4.000000,1
991,Babyville,3.500000,1


## 2.3 Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [11]:
table


['books', 'ratings', 'reviews', 'publishers', 'authors']

In [12]:
taskfunc(
'''
SELECT 
    publishers.publisher,
    COUNT(books.book_id)
FROM
    publishers
    
    LEFT JOIN 
       books ON books.publisher_id = publishers.publisher_id
WHERE
books.num_pages > 50

    
Group by
publishers.publisher_id
ORDER BY
count DESC



LIMIT 1
'''
)

,publisher,count
0,Penguin Books,42


## 2.4 Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [13]:
table

['books', 'ratings', 'reviews', 'publishers', 'authors']

In [14]:
taskfunc(
'''
SELECT 
    authors.author,
    AVG(ratings.rating),
    COUNT(ratings.rating)
FROM
    books
    
    LEFT JOIN 
       authors ON authors.author_id = books.author_id
    LEFT JOIN 
       ratings ON ratings.book_id = books.book_id


    
Group by
authors.author
HAVING
Count(ratings.rating) > 50
ORDER BY
avg DESC



LIMIT 1
'''
)

,author,avg,count
0,J.K. Rowling/Mary GrandPré,4.288462,312


## 2.5 Find the average number of text reviews among users who rated more than 50 books.

In [15]:
table

['books', 'ratings', 'reviews', 'publishers', 'authors']

In [16]:
taskfunc(
'''
SELECT 
            count(username),
            username
FROM ratings

group by 
username
ORDER BY
COUNT DESC

LIMIT 6
'''
)

,count,username
0,56,martinadam
1,56,paul88
2,55,richard89
3,55,sfitzgerald
4,53,jennifermiller
5,51,xdavis


In [17]:
taskfunc(
'''
SELECT 
    reviews.username,
    reviews.text
    
FROM
    reviews
    
    LEFT JOIN 
       ratings ON ratings.username = reviews.username
   
WHERE
reviews.username  IN ('martinadam','paul88','richard89','sfitzgerald','jennifermiller','xdavis')
Group by
reviews.text,
reviews.username



'''
)

,username,text
0,xdavis,Hold Mr stock factor officer community actuall...
1,jennifermiller,Appear give matter wrong. Individual informati...
2,xdavis,Interesting experience suggest herself mind. W...
3,richard89,Church structure rather understand. Attorney b...
4,richard89,Design letter against billion position condition.
...,...,...
141,xdavis,Rise ability expert bank individual. Practice ...
142,sfitzgerald,Mouth provide oil agency occur wear year air. ...
143,richard89,Measure live lead difficult modern door. Inter...
144,xdavis,Sound standard structure light no success. Mee...


In [18]:
display('The Average is', 146/6)

'The Average is'

24.333333333333332

solve it in a subquery for a clean code

In [19]:
taskfunc(
'''
SELECT

    count(text) / count(DISTINCT(reviews.username))
From 
    reviews
LEFT JOIN 
    (
    SELECT 
        DISTINCT ratings.username
    FROM 
        ratings
    Group by 
        username
    HAVING
        count(username) > 50
    ) AS SUBQ
ON SUBQ.username = reviews.username
WHERE
    SUBQ.username = reviews.username 


'''
)

,?column?
0,24
